In [1]:
import pandas as pd
import numpy as np
from scipy import optimize

In [2]:
df = pd.read_csv('possum.csv')

In [18]:
df.head()
df = df.dropna(subset=['age', 'hdlngth', 'skullw', 'totlngth', 'taill'])

In [19]:
features = ['hdlngth', 'skullw', 'totlngth', 'taill']
X = np.array(df[features])
y = np.array(df['age'])

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [21]:
X_scaled = np.c_[np.ones(X_scaled.shape[0]), X_scaled]

In [22]:
def f(beta):
    return X_scaled@beta - y
    

In [26]:
beta_0 = 0.1 * np.array([1, 1, 1, 1, 1])
beta = optimize.least_squares(f, beta_0)

In [33]:
beta.x

array([ 1.895393  ,  2.06051363,  1.34168125,  0.78667356, -0.06681448])

In [31]:
# consider an linear model y = X*beta
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_scaled, y)
reg.coef_


array([ 0.        ,  2.06051347,  1.34168141,  0.7866734 , -0.06681415])

In [34]:
reg.intercept_

1.8953929638857794

In [50]:
# consider an exponential model y = A*e^(Xw)
# subject to sum(w)=1, 0<=w<=1
def g(w):
    return 0.5 * np.linalg.norm(X_scaled@w - np.log(y))**2

In [52]:
from scipy.optimize import LinearConstraint, Bounds
w_0 = 0.1 * np.array([1, 1, 1, 1, 1])
equality_conatraints = LinearConstraint([[1, 1, 1, 1, 0]], 
                                        [0], [1])
bounds = Bounds([0, 0, 0, 0, -np.inf], [1, 1, 1, 1, np.inf])
w = optimize.minimize(g, w_0,
                      constraints=equality_conatraints,
                      bounds=bounds)

In [60]:
w

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 15.143357531840307
       x: [ 5.729e-01  4.271e-01  1.581e-14  1.575e-14  8.342e-01]
     nit: 5
     jac: [-3.683e+00 -3.683e+00 -2.868e+00 -2.932e+00 -9.537e-07]
    nfev: 32
    njev: 5